# Proyecto 1 del curso de Machine Learning
## Presentado por Daniel Mateo Guatibonza [201611360] y Sofía Gutiérrez R [201612121]

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os, zipfile, warnings, types, pathlib
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import tensorflow as tf
import IPython.display as display
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras import layers

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
data_dir = pathlib.Path("./DataNous/Facil_Pocos")
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*')])
image_count = len(list(data_dir.glob('*/*.png')))

In [4]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
BATCH_SIZE = 32
IMG_HEIGHT = 64
IMG_WIDTH = 64
NB_CLASSES = len(CLASS_NAMES)
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES))

Found 320 images belonging to 2 classes.


In [ ]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(16):
        ax = plt.subplot(4,4,n+1)
        plt.imshow(image_batch[n])
        plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
        plt.axis('off')

In [ ]:
image_batch, label_batch = next(data_gen)
show_batch(image_batch, label_batch)

In [6]:
ds_mario = tf.data.Dataset.from_generator(lambda: data_gen,
                     output_types=(tf.float32, tf.float32),
                     output_shapes=([BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH, 3],
                                    [BATCH_SIZE, NB_CLASSES]))

In [ ]:
for i in ds_mario.take(1):
    plt.imshow(i[0][0])
    print(i[1])

In [7]:
initial_epochs = 10
train_size = int(0.8 * image_count)
val_size = int(0.1 * image_count)
test_size = int(0.1 * image_count)
num_batches = train_size / BATCH_SIZE

train_dataset = ds_mario.take(train_size)
test_dataset = ds_mario.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

train_dataset = train_dataset.repeat(initial_epochs)

In [ ]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3) 
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
base_model.trainable = False
base_model.summary()

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = tf.keras.layers.Dense(NB_CLASSES)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
initial_epochs = 10
validation_steps=20
loss0,accuracy0 = model.evaluate(val_dataset, steps = validation_steps)

In [ ]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=val_dataset)

In [ ]:
model.save('modelo_entrenado.h5')
modelo = tf.keras.models.load_model('modelo_entrenado.h5')
modelo.summary()

In [ ]:
loss, accuracy = modelo.evaluate(val_dataset, steps = validation_steps)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
base_model.trainable = True
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])
model.summary()
len(model.trainable_variables)

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch =  history.epoch[-1],
                         validation_data=validation_batches)

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# Red Sofi

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, BatchNormalization

In [ ]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3) 

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=IMG_SHAPE))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.1))


model.add(Conv2D(64, (3, 3), input_shape=IMG_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.1))


model.add(Conv2D(128, (3,3), input_shape=IMG_SHAPE))
model.add(Activation('relu'))
model.add(Dropout(rate=0.1))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

# Dropout
model.add(Dropout(rate=0.5))
    
model.add(Dense(32, activation='relu'))
    

model.add(Dense(NB_CLASSES))

# Cambiar!!!
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
history = model.fit(train_dataset,
                                epochs=initial_epochs,
                                validation_data=val_dataset)

# Modelo con ayuda de Cristian

In [11]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3) 
model_cristian = tf.keras.Sequential()
model_cristian.add(tf.keras.layers.Conv2D(4, (3,3), strides=(2,2), padding='same', input_shape=IMG_SHAPE))
model_cristian.add(tf.keras.layers.Conv2D(8, (3,3), strides=(2,2), padding='same'))
model_cristian.add(tf.keras.layers.Conv2D(16, (3,3), strides=(2,2), padding='same'))
model_cristian.add(tf.keras.layers.Flatten())
model_cristian.add(tf.keras.layers.Dense(NB_CLASSES))
model_cristian.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 4)         112       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 8)         296       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 16)          1168      
_________________________________________________________________
flatten_2 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2050      
Total params: 3,626
Trainable params: 3,626
Non-trainable params: 0
_________________________________________________________________


In [12]:
base_learning_rate = 0.0001
initial_epochs = 10
model_cristian.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
                          loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                          metrics=['accuracy'])
history = model_cristian.fit(train_dataset,
                                epochs=initial_epochs,
                                validation_data=val_dataset,
                                steps_per_epoch=num_batches)
model_cristian.summary()

Train for 8.0 steps
Epoch 1/10
8/8 [==============================] - 994s 124s/step - loss: 0.6780 - accuracy: 0.5020


KeyboardInterrupt: 

In [ ]:
model_cristian.save('modelo_cristian_entrenado.h5')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()